<a href="https://colab.research.google.com/github/RehanFastians/FakeNewsDetector/blob/main/FakeNewsTesting(CLI%26GUI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit pyngrok transformers requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import requests
from transformers import pipeline

# === Config ===
GNEWS_API_KEY = "389981ede1c11322cc318e2bbbc34e0f"  # ← Replace this
GNEWS_URL = "https://gnews.io/api/v4/search"

# === Load Classifier ===
@st.cache_resource
def load_model():
    return pipeline(
        "text-classification",
        model="jy46604790/Fake-News-Bert-Detect",
        tokenizer="jy46604790/Fake-News-Bert-Detect"
    )

classifier = load_model()

# === GNews Search ===
def search_gnews(statement):
    params = {'q': statement, 'token': GNEWS_API_KEY, 'lang': 'en', 'max': 5}
    try:
        res = requests.get("https://gnews.io/api/v4/search", params=params)
        return res.json().get("articles", [])
    except Exception as e:
        return []

# === App UI ===
st.title("📰 Fake News Detector")
st.markdown("Enter a news statement to classify it as **REAL** or **FAKE** with confidence, and see real-time news support.")

user_input = st.text_input("🔎 Enter a news claim:")

if user_input:
    result = classifier(user_input[:500])[0]
    label = "REAL" if result["label"] == "LABEL_1" else "FAKE"
    confidence = round(result["score"], 2)

    st.subheader("📊 Prediction by Model")
    st.write(f"**Label**: `{label}`")
    st.write(f"**Confidence**: `{confidence*100}%`")

    # Fetch News
    articles = search_gnews(user_input)
    if articles:
        st.subheader("📰 Related News Articles")
        for article in articles:
            st.markdown(f"- [{article['title']}]({article['url']})")
    else:
        st.warning("No related news found.")

    # Verdict
    st.subheader("🧠 Verdict by internet surfing")
    if label == "FAKE" and articles:
        st.info("⚠️ Likely FAKE, but some similar news found. Cross-check.")
    elif label == "FAKE":
        st.error("❌ FAKE — no supporting real news found.")
    elif label == "REAL" and confidence > 0.9:
        st.success("✅ REAL — high confidence.")
    else:
        st.info("✅ Possibly REAL — verify using relevant sources.")


Writing app.py


In [ ]:
from pyngrok import ngrok, conf
import os

# Set your authtoken
conf.get_default().auth_token = "2zX6PaLPBsPyILCnaeaTRPzEzZX_2kYNkYsRHVaqsobeAgMTa"

# Kill previous tunnels (if any)
ngrok.kill()

# Start tunnel
public_url = ngrok.connect(addr="8501", proto="http")

print("🔗 Your Streamlit app is live at:", public_url)

# Launch Streamlit
!streamlit run app.py &> /dev/null &

🔗 Your Streamlit app is live at: NgrokTunnel: "https://1c6e1f7c77f3.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill()